# Simulated Annealing

### Classe para Leitura das Instâncias

In [1]:
import time

import numpy as np



class Instance:
    def __init__(self, filename):

        path = f"{filename}"

        f = open(path, "r")

        self.num_items, self.num_forfeits_pairs, self.budget = map(
            int, f.readline().split(" ")
        )

        f.close()

        # items definition
        items = []
        for i in range(self.num_items):
            items.append(i)

        self.items = np.array(items)

        line_counter = 1

        self.profits = np.loadtxt(
            path, delimiter=" ", skiprows=line_counter, max_rows=1
        )
        line_counter += 1
        
        self.weights = np.loadtxt(
            path, delimiter=" ", skiprows=line_counter, max_rows=1
        )
        line_counter += 1

        max = 2 * self.num_forfeits_pairs

        self.forfeit_cost_and_forfeits_pairs = np.loadtxt(
            path, delimiter=" ", skiprows=line_counter, max_rows=max, usecols=(0, 1)
        ).tolist()

        self.forfeits_costs = [
            v for i, v in enumerate(self.forfeit_cost_and_forfeits_pairs) if i % 2 == 0
        ]

        self.forfeits_pairs = [
            v for i, v in enumerate(self.forfeit_cost_and_forfeits_pairs) if i % 2 != 0
        ]

        # remove the first element of the sublist
        for i in range(len(self.forfeits_costs)):
            self.forfeits_costs[i].pop(0)

        # transform list of lists into a single list - flatten
        self.forfeits_costs = [
            item for sublist in self.forfeits_costs for item in sublist
        ]

        self.forfeits_costs = np.array(self.forfeits_costs)
        self.forfeits_pairs = np.array(self.forfeits_pairs)      

# Algoritmo Construtivo

In [2]:
import random as rd

def greedyalgorithm(items, weights, profits, budget, alpha, mD):    
    
    cap=budget    

    #print(sorted_items)    
    solution=[]
    scost=[]
    sweights=[]
    cost=0
    index=0
    
    if alpha == 0:  # totalmente aleatório 
        remaining_items=zip(items,weights,profits)
        sorted_items=sorted(remaining_items, key= lambda x:x[2]/(x[1]+1), reverse=True)
        #print(tuple(items))        
        
        rd_index=rd.choice(range(0,len(sorted_items))) 
        candidate=sorted_items[rd_index][0]
        
        while cap-weights[candidate]>0:                                        
                cap=cap-weights[candidate]
                solution.append(candidate) 
                
                
                penalidade=sum(mD[candidate][solution])                
                #if penalidade>0: print(f"Penalidade do Item {sorted_items[rd_index][0]} é {penalidade}")
                    
                cost=cost+profits[candidate]-penalidade
                
                scost.append(profits[candidate]-penalidade)
                
                sweights.append(weights[candidate])
                
                sorted_items.remove(sorted_items[rd_index])
                rd_index=rd.choice(range(0,len(sorted_items)))  
                candidate=sorted_items[rd_index][0]
        
                    
    else:
            if alpha == 1: #totalmente guloso

                remaining_items=zip(items,weights,profits)
                sorted_items=sorted(remaining_items, key= lambda x:x[2]/(x[1]+1), reverse=True)
                candidate=sorted_items[0][0]

                while cap-weights[candidate]>0:                                                  
                    
                    cap=cap-weights[candidate]
                    solution.append(candidate)  
                    
                    penalidade=sum(mD[candidate][solution])                
                    
                    cost=cost+profits[candidate]-penalidade
                    
                    scost.append(profits[candidate]-penalidade)
                    
                    sweights.append(weights[candidate])
                    
                    sorted_items.remove(sorted_items[0])
                    
                    if len(sorted_items)>=0:
                        candidate=sorted_items[0][0] 
                    
            else: # semi-guloso
                remaining_items=zip(items,weights,profits,profits/weights)
                sorted_items=sorted(remaining_items, key= lambda x:x[2]/(x[1]+1), reverse=True)
                
                
                # o cálculo dos limites da lcr
                hmax=sorted_items[0][3]
                hmin=sorted_items[-1][3]

                    #ub = hmax + alpha * (hmin - hmax)
                lb = hmin                            
                ub=hmax + alpha * (hmin - hmax)                  

                    #lista restrita de candidatos
                    #lcr=list(filter(lambda x: x[3]<=ub and x[3] >= lb, sorted_items))                                
                lcr=list(filter(lambda x: x[3]>=ub, sorted_items))
                rd_index=rd.choice(range(0,len(lcr))) 
                candidate=lcr[rd_index][0]
                
                #print(candidate)
                while cap-weights[candidate]>0:  
                                                                          
                    cap=cap-weights[candidate]
                    solution.append(candidate)    
                    #print(f" Solution: {solution}")
                                        
                    
                    #penalidade
                    penalidade=sum(mD[candidate][solution])                
                    #if penalidade>0: print(f"Penalidade do Item {sorted_items[rd_index][0]} é {penalidade}")
                                        
                    cost=cost+profits[candidate]-penalidade                                        
                    
                    scost.append(profits[candidate]-penalidade)
                    
                    sweights.append(weights[candidate])
                
                    #removendo o item já inserido
                    sorted_items.remove(sorted_items[sorted_items.index(lcr[rd_index])])                    
               
                                          
                    #sorted_items.sort(key= lambda x:(x[2]-sum(mD[x[0]][solution]))/(x[1]+1), reverse=True)                                        

                     # o cálculo dos limites da lcr
                    hmax=sorted_items[0][3]
                    hmin=sorted_items[-1][3]
                    
                    #ub = hmax + alpha * (hmin - hmax)
                    lb = hmin                            
                    ub=hmax + alpha * (hmin - hmax)
                 
                    #lista restrita de candidatos                                
                    lcr=list(filter(lambda x: x[3]>=ub, sorted_items))

                    #print(f"LCR: {lcr}")
                    rd_index=rd.choice(range(0,len(lcr))) 
                    candidate=lcr[rd_index][0]
                                
    
    return solution,cost,cap,scost,sweights,sorted_items

# Matriz de Penalidades

In [3]:
def calculate_penalty(items,forfeits_pairs, forfeits_costs):
    mD=np.zeros((int(len(items)),int(len(items))))       
    
    for index, pair in enumerate(forfeits_pairs):
        mD[int(pair[0]),int(pair[1])]=forfeits_costs[index]                
    return mD

# Simulated Annealing

### Get neighbor solution - Test 1

Walk through viable solution

In [32]:
def swap(solution:set, in_items, out_items) -> set:
  
  # choose random element from a set
  enter_item = random.choice(in_items)
  leave_item = random.choice(out_items)

  # exchange in solution
  solution.remove(leave_item)
  solution.add(enter_item)

  #update exchange items for the next iteration
  in_items.remove(enter_item)
  out_items.remove(leave_item)

  return solution


def get_neighbor(solution, items, weights, budget): 
    temp_weight = 0
    solution_weights = 0
    solution_set = set(solution)
    items_set = set(items)
    remaining_items = items_set.difference(set(solution))
    in_items = list()
    out_items = list()

    for item in solution_set:
        solution_weights += weights[item]

    for leave_item in solution_set: 
        for enter_item in remaining_items:

            temp_weight = solution_weights - weights[leave_item] + weights[enter_item]

            if temp_weight <= budget: #feasibility
                in_items.append(enter_item)
                out_items.append(leave_item)

    neighbor_solution = swap(solution_set, in_items, out_items)
    return neighbor_solution # return when the first feasible  neighbor solution is found


### Get neighbor - Test 2

Walk through inviable solution

In [33]:
# def swap(solution:set, enter_item, leave_item) -> set:
  
#   # exchange in solution
#   solution.remove(leave_item)
#   solution.add(enter_item)

#   return solution


# def get_neighbor(solution, items, weights, budget): 
#     temp_weight = 0
#     solution_weights = 0
#     solution_set = set(solution)
#     items_set = set(items)
#     remaining_items = items_set.difference(set(solution))
    

#     enter_item = random.choice(list(remaining_items))
#     leave_item = random.choice(list(solution_set))

#     neighbor_solution = swap(solution_set, enter_item, leave_item)
    
#     return neighbor_solution # return when the first feasible  neighbor solution is found


### Get solution cost

In [34]:
def get_solution_cost(solution, profits, forfeits_pairs, mD) -> int:
    forfeits_pairs_set = {tuple(x) for x in forfeits_pairs}

    cost = 0

    for item in solution:
        cost += profits[item]

    for i in solution:
        for j in solution:
            if i > j:
                if (i, j) in forfeits_pairs_set:
                    cost -= mD[i][j]

    return cost

### Simulated Annealing Function

In [35]:
import random
import math

def sa(max_iter, T, t_initial, beta, items, weights, profits, budget, forfeits_costs, forfeits_pairs, alpha, mD):
    # generate initial solution
    s_current, *_ = greedyalgorithm(items, weights, profits, budget, alpha, mD)
    const_cost = get_solution_cost(s_current, profits, forfeits_pairs, mD)
    print(f'constructive cost: {const_cost}')
    s_best = s_current
    iter = 0
    t = t_initial
    while t > T:
        while iter < max_iter:
            iter = iter + 1
            s_neighbor = get_neighbor(s_current, items, weights, budget)
            # print(f"difference generate neighbor: {set(s_neighbor).difference(set(s_current))}")
            delta = get_solution_cost(s_neighbor, profits, forfeits_pairs, mD) - get_solution_cost(s_current, profits,forfeits_pairs, mD)
            if delta > 0:
                s_current = s_neighbor
                if get_solution_cost(s_current, profits, forfeits_pairs, mD) > get_solution_cost(s_best, profits, forfeits_pairs, mD):
                    s_best = s_current 
            else:
                if random.random() < math.exp(-(delta/t)):
                    s_current = s_neighbor
            
        t = beta * t
        iter = 0
    return s_best






### Main

In [39]:
filename = "kpf_1.txt"
print(filename)
problem_instance = Instance(filename)

mD=calculate_penalty(problem_instance.items,problem_instance.forfeits_pairs, problem_instance.forfeits_costs)
start_time = time.time()
alpha=1 # greedy constructive
max_iter = 200
T = 5
t_initial = 200
beta = 0.5 # cooling rate

start_time = time.time()

solution_sa = sa(
  max_iter, 
  T, 
  t_initial, 
  beta, 
  problem_instance.items, 
  problem_instance.weights, 
  problem_instance.profits, 
  problem_instance.budget, 
  problem_instance.forfeits_costs, 
  problem_instance.forfeits_pairs, 
  alpha,
  mD
  )

# print(f"Custo construtivo: {cost_greedy}")

end_time = time.time()

wall_time = end_time - start_time


print(f"Time: {wall_time}")
print(f"Solution Simulated Annealing: {solution_sa}")

s_constructive, *_ = greedyalgorithm(problem_instance.items, 
  problem_instance.weights, 
  problem_instance.profits, 
  problem_instance.budget, 
  alpha, 
  mD)

# print(f"difference: {set(s_constructive).difference(set(solution_sa))}")

cost_sa = get_solution_cost(solution_sa, problem_instance.profits, problem_instance.forfeits_pairs, mD)
print(f"Cost Simulated Annealing: {cost_sa}")


kpf_1.txt
constructive cost: 1748.0
Time: 87.8301477432251
Solution Simulated Annealing: {4, 5, 8, 9, 11, 14, 16, 17, 18, 19, 21, 22, 24, 29, 31, 35, 37, 38, 39, 40, 41, 42, 44, 47, 52, 54, 55, 56, 59, 60, 63, 66, 72, 73, 77, 79, 82, 87, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 101, 102, 105, 107, 108, 109, 111, 116, 117, 127, 128, 130, 134, 135, 138, 140, 142, 145, 146, 148, 150, 151, 154, 156, 157, 159, 160, 161, 162, 163, 164, 170, 171, 173, 174, 175, 177, 178, 181, 182, 186, 187, 188, 192, 195, 196, 199, 201, 202, 203, 204, 209, 211, 213, 219, 220, 224, 225, 226, 227, 229, 232, 234, 237, 239, 240, 241, 244, 245, 246, 247, 248, 250, 253, 254, 256, 257, 262, 264, 265, 266, 269, 271, 288, 290, 293, 295, 296, 297, 299, 300, 305, 306, 309, 310, 311, 312, 314, 315, 317, 327, 328, 329, 337, 344, 346, 348, 353, 356, 359, 362, 363, 364, 366, 367, 368, 370, 373, 378, 380, 382, 386, 389, 395, 396, 402, 403, 406, 409, 411, 412, 413, 416, 417, 423, 425, 427, 434, 437, 440, 446, 450, 452, 454